In [1]:
import pandas as pd
import numpy as np
import regex
from pathlib import Path

In [2]:
HRSA_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\hrsa\HRSA - 2021 - Cleaned.csv')

In [3]:
# This dataset was created 5-27-2021 by HRSA
# tx_center Count - 254
print(HRSA_raw['OTC Name'].nunique())
# State Count - 48
print(HRSA_raw['Common State Name'].nunique())
# Cities Count - 137
print(HRSA_raw['OTC City'].nunique())
print(HRSA_raw.info())

254
48
137
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809 entries, 0 to 808
Data columns (total 30 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   OPO Name                                               809 non-null    object 
 1   OTC Name                                               809 non-null    object 
 2   OTC Service List                                       809 non-null    object 
 3   OPO Provider #                                         809 non-null    object 
 4   OPO Address                                            809 non-null    object 
 5   OPO City                                               809 non-null    object 
 6   OPO State                                              809 non-null    object 
 7   OPO ZIP                                                809 non-null    object 
 8   OPO Telephone #                        

In [4]:
# Master list of OTC 2021
otc_2021 = HRSA_raw[['OTC Name', 'OTC Address', 'OTC City', 'Common State Name', 'Common State Abbreviation', 'OTC ZIP', 'OTC Longitude', 'OTC Latitude', 'OTC Code']]
otc_2021 = otc_2021.drop_duplicates().reset_index(drop=True)

#OPO master list
opo = HRSA_raw[['OPO Name', 'OPO Address', 'OPO City', 'OPO State', 'OPO ZIP', 'OPO Telephone #', 'OPO Provider #']]
opo = opo.drop_duplicates().reset_index(drop=True)

# Organs transplanted from each OTC
otc_service = HRSA_raw[['OTC Code', 'OTC Service List', 'Organ Transplantation Center Service Type Description']]

# Regions Lists
regions = HRSA_raw[['Common Region Name', 'Common State Name', 'Common Region Code', 'OPO Provider #', 'OTC Code']]
regions = regions.drop_duplicates().reset_index(drop=True)

opo_regions = regions[['Common Region Code', 'OPO Provider #']]
opo_regions = opo_regions.drop_duplicates().reset_index(drop=True)

otc_regions = regions[['Common Region Code', 'OTC Code']]
otc_regions = otc_regions.drop_duplicates().reset_index(drop=True)

regions = regions.drop(columns=['OPO Provider #', 'OTC Code'])
regions = regions.drop_duplicates().reset_index(drop=True)

# OPO key = OPO ZIP
# OTC key = OTC ZIP
hrsa_census_keys = HRSA_raw[['Common State FIPS Code', 'Common State County FIPS Code', 'Common County Name', 'Common City Name with State Abbreviation', 'Common Congressional District Name', 'Common State FIPS Code Congressional District Number', 'Common Postal Code']]
hrsa_census_keys = hrsa_census_keys.drop_duplicates()

srtr_raw['OTC Address'] = srtr_raw['OTC Address'].astype(object)
srtr_raw['Common State Name'] = srtr_raw['Common State Name'].astype(object)
srtr_raw['OTC ZIP'] = srtr_raw['OTC ZIP'].astype(object)
otc_2021['OTC ZIP'] = otc_2021['OTC ZIP'].astype(object)

srtr_raw.merge(otc_2021, how='inner', on='OTC Code')

hospitals = [srtr_raw, otc_2021]
pd.concat(hospitals, join='inner', keys='OTC Code')

In [5]:
srtr_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\otc-2009-raw.csv')

In [6]:
srtr_pending = srtr_raw.loc[srtr_raw['OTC Code'].isnull()].reset_index(drop=True)
srtr_finished = srtr_raw.loc[srtr_raw['OTC Code'].notnull()].reset_index(drop=True)

In [7]:
srtr_pending_manual = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\srtr_pending_manual.csv')

In [8]:
srtr_pending_2 = srtr_pending_manual.loc[srtr_pending_manual['OTC Code'].isnull()].reset_index(drop=True)
srtr_finished_2 = srtr_pending_manual.loc[srtr_pending_manual['OTC Code'].notnull()].reset_index(drop=True)

In [9]:
finisheds = [srtr_finished, srtr_finished_2]

In [10]:
srtr_pending_2.head(1)
# Drop Common State Name
srtr_pending_2 = srtr_pending_2.drop(columns=['Common State Name'])

In [11]:
srtr_pending_2.head(1)

,OTC Name,OTC Address,OTC City,Common State Abbreviation,OTC ZIP,OTC Longtitude,OTC Latitude,OTC Code
0,Baptist Medical Center Princeton,NaN,Birmingham,AL,NaN,NaN,NaN,NaN


In [12]:
us_hospitals_full = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\us_hospital_locations.csv')

In [13]:
us_hospitals = us_hospitals_full[['NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'LONGITUDE', 'LATITUDE']]

In [14]:
srtr_finished_2 = srtr_pending_2.drop(columns=['OTC Code'])

In [15]:
srtr_pending_2['OTC Name'] = srtr_pending_2['OTC Name'].str.upper()

In [16]:
us_hospitals.NAME = us_hospitals.NAME.str.upper()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
us_hospitals = us_hospitals.rename(columns={'NAME' : 'OTC Name', 'ADDRESS' : 'OTC Address', 'CITY' : 'OTC City', 'STATE' : 'Common State Abbreviation', 'ZIP' : 'OTC ZIP', 'LONGITUDE' : 'OTC Longtitude', 'LATITUDE' : 'OTC Latitude'})

In [18]:
srtr_pending_2['OTC City'] = srtr_pending_2['OTC City'].str.upper()
srtr_pending_2['key'] = srtr_pending_2['OTC Name'] + srtr_pending_2['OTC City'] + srtr_pending_2['Common State Abbreviation']
us_hospitals['key'] = us_hospitals['OTC Name'] + us_hospitals['OTC City'] + us_hospitals['Common State Abbreviation']

In [19]:
pd.set_option('display.max_row', 10)


In [20]:
srtr_pending_2.head(1)

,OTC Name,OTC Address,OTC City,Common State Abbreviation,OTC ZIP,OTC Longtitude,OTC Latitude,OTC Code,key
0,BAPTIST MEDICAL CENTER PRINCETON,NaN,BIRMINGHAM,AL,NaN,NaN,NaN,NaN,BAPTIST MEDICAL CENTER PRINCETONBIRMINGHAMAL


In [21]:
us_hospitals.head(1)

,OTC Name,OTC Address,OTC City,Common State Abbreviation,OTC ZIP,OTC Longtitude,OTC Latitude,key
0,CENTRAL VALLEY GENERAL HOSPITAL,1025 NORTH DOUTY STREET,HANFORD,CA,93230,-119.645667,36.336159,CENTRAL VALLEY GENERAL HOSPITALHANFORDCA


In [22]:
test = srtr_pending_2[['key']]

In [23]:
test2 = test.merge(us_hospitals, how='left', on='key')
# Drop key, drop/fix OTC name column

In [24]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 98
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   key                        99 non-null     object 
 1   OTC Name                   21 non-null     object 
 2   OTC Address                21 non-null     object 
 3   OTC City                   21 non-null     object 
 4   Common State Abbreviation  21 non-null     object 
 5   OTC ZIP                    21 non-null     float64
 6   OTC Longtitude             21 non-null     float64
 7   OTC Latitude               21 non-null     float64
dtypes: float64(3), object(5)
memory usage: 7.0+ KB


In [25]:
 pending3 = test2.loc[test2['OTC ZIP'].isnull()].reset_index()

In [26]:
pending3

,index,key,OTC Name,OTC Address,OTC City,Common State Abbreviation,OTC ZIP,OTC Longtitude,OTC Latitude
0,0,BAPTIST MEDICAL CENTER PRINCETONBIRMINGHAMAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,JOHN L. MCCLELLAN VETERANS ADMINISTRATION HOSP...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,PHOENIX REGIONAL MEDICAL CENTERPHOENIXAZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,VETERANS ADMINISTRATION MEDICAL CENTER TUCSONT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,ALTA BATES MEDICAL CENTERBERKELEYCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
73,94,SALT LAKE CITY VETERANS ADMINISTRATION MEDICAL...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,95,FLETCHER ALLEN HEALTH CAREBURLINGTONVT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,96,JOHN L DOYNE HOSPMILWAUKEEWI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,97,ZABLOCKI VETERANS ADMINISTRATION MEDICAL CENTE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
%who

HRSA_raw	 Path	 finisheds	 hrsa_census_keys	 np	 opo	 opo_regions	 os	 otc_2021	 
otc_regions	 otc_service	 pd	 pending3	 regex	 regions	 site	 srtr_finished	 srtr_finished_2	 
srtr_pending	 srtr_pending_2	 srtr_pending_manual	 srtr_raw	 sys	 test	 test2	 us_hospitals	 us_hospitals_full	 



In [ ]:
still_looking = 